In [7]:
import mlflow
import os

# 强制删除可能存在的干扰变量
if "AWS_PROFILE" in os.environ:
    del os.environ["AWS_PROFILE"]

# 再次确认一下你的 AK SK 是否真的被 Python 读到了（打出来前几位看看）
# 如果这里打印出来是 None，说明凭证根本没配进去
print("AK:", os.environ.get("AWS_ACCESS_KEY_ID"))
print("Region:", os.environ.get("AWS_DEFAULT_REGION"))

TRACKING_SERVER_HOST = "ec2-18-181-238-89.ap-northeast-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

AK: None
Region: None


In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-18-181-238-89.ap-northeast-1.compute.amazonaws.com:5000'


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, name="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/11/19 08:10:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


default artifacts URI: 's3://huangmlops/1/f36fea7990f34c888e4eff30a9a02f1c/artifacts'
🏃 View run mysterious-sow-897 at: http://ec2-18-181-238-89.ap-northeast-1.compute.amazonaws.com:5000/#/experiments/1/runs/f36fea7990f34c888e4eff30a9a02f1c
🧪 View experiment at: http://ec2-18-181-238-89.ap-northeast-1.compute.amazonaws.com:5000/#/experiments/1


In [9]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [10]:
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/11/19 08:12:27 WARNING mlflow.tracking._model_registry.fluent: Run with id f36fea7990f34c888e4eff30a9a02f1c has no artifacts at artifact path 'models', registering model based on models:/m-6068cbce8f9c480a81d34754ff5687ca instead
2025/11/19 08:12:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1763539947849, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1763539947849, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='f36fea7990f34c888e4eff30a9a02f1c', run_link='', source='models:/m-6068cbce8f9c480a81d34754ff5687ca', status='READY', status_message=None, tags={}, user_id='', version='1'>